# **Setting up API Keys**

In [1]:
from google.colab import userdata
PINCECONE_API_KEY = userdata.get('PINCECONE_API_KEY')

In [13]:
import os
os.environ["PINECONE_API_KEY"] = PINCECONE_API_KEY

In [2]:
# Gemini API setup
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:embedContent"


# **Installing and Importing libraries**

In [ ]:
!pip install -qq pinecone
!pip install -q langchain langchain-community langchain-text-splitters requests
!pip install -qq pypdf
!pip install --upgrade langchain langchain_community

In [4]:
from langchain_core.embeddings import Embeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import requests
from typing import List

In [ ]:
!pip show pinecone

# **Raw pinecone client for debugging purposes ( we will use langchain for it)**

In [21]:
from pinecone import Pinecone

# Initialize client
pc = Pinecone(api_key=PINCECONE_API_KEY)

# Connect to your existing index
index = pc.Index("test")

print(index.describe_index_stats())

{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


# **Data loading and prepration**

In [5]:
!mkdir pdfs

In [ ]:
!gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
!gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

In [7]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [ ]:
text_chunks

In [13]:

len(text_chunks)

168

In [ ]:
text_chunks[1]

# **Gemini Embedding Setup**

In [9]:
# Custom Gemini Embedding class
class GeminiEmbedding(Embeddings):
    def __init__(self, api_key: str):
        self.api_key = api_key

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._embed(text) for text in texts]

    def embed_query(self, text: str) -> List[float]:
        return self._embed(text)

    def _embed(self, text: str) -> List[float]:
        headers = {"Content-Type": "application/json"}
        params = {"key": self.api_key}
        payload = {"model": "models/gemini-embedding-001", "content": {"parts":[{"text": text}]}}
        response = requests.post(GEMINI_API_URL, json=payload, headers=headers, params=params)

        if response.status_code != 200:
            raise Exception(f"Gemini API error: {response.text}")

        data = response.json()
        return data["embedding"]["values"]

# Setup
embedding = GeminiEmbedding(api_key=GEMINI_API_KEY)


# **Embedding and storing in pinecone using langchain**

In [ ]:
!pip install -q langchain-pinecone


In [11]:
from langchain_pinecone import PineconeVectorStore

In [23]:
vectorstore = PineconeVectorStore.from_documents(
    documents=text_chunks,     # your split PDF docs
    embedding=embedding,
    index_name="test"          # same index you created
)


# **If you have done everything before and want to connect and query whenever, without re-embedding everything.**

In [14]:
# Loading the index
vectorstore = PineconeVectorStore.from_existing_index(
    index_name="test",
    embedding=embedding
)

In [24]:
print(index.describe_index_stats())

{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 168}},
 'total_vector_count': 168,
 'vector_type': 'dense'}


# **RAG pipeline**

In [ ]:
!pip install -q langchain-google-genai

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [17]:
# 1. Create retriever from Pinecone
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print(retriever.search_type)        # should be 'similarity'
print(retriever.search_kwargs)      # {'k': 3}

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GEMINI_API_KEY
)

similarity
{'k': 3}


In [18]:
# 2. Build RetrievalQA chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [21]:
# 3. Ask a question

query1 = "What is YOLOv7 about?"
query2 = "What is my name?" # For testing like Gemini is actually citing the documents or not
response = qa_chain.invoke({"query": query1})

print("Answer:", response["result"])
print("\nSources:")
for doc in response["source_documents"]:
    print("-", doc.metadata.get("source"))

Answer: YOLOv7 is a real-time object detector that surpasses other detectors in both speed and accuracy, achieving the highest accuracy (56.8% AP) among known detectors in the 5 FPS to 160 FPS range.  It was trained only on the MS COCO dataset without using other datasets or pre-trained weights.  Compared to YOLOv5-X, YOLOv7-X reduces parameters and computation while improving accuracy.

Sources:
- pdfs/yolov7paper.pdf
- pdfs/yolov7paper.pdf
- pdfs/yolov7paper.pdf
